# Coursera IBM Data Science Professional / Data Science Capstone 

## Final assignment part 1: Getting data from Wikipedia

This Jupyter notebook is for the Coursera Data Science Capstone project: Segmenting and Clustering Neighborhoods in Toronto.
The first part is about getting the data from the Toronto neighbourhoods by scraping a wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

However, first things first. Let's import all the relevant libraries and packages needed.

In [40]:
import pandas as pd
import numpy as np
import requests
import io

Now, I'm requesting the data from the wikipedia page.

In [41]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

And insert the data directly into a dataframe by using pandas read_html function

In [42]:
df_toronto = pd.read_html(website_url, header = 0)[0]

In [43]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Now I'm deleting all boroughs that are not assigned (according to the course assignment: "Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned."

In [44]:
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Neighbourhoods that have the same postcode but several boroughs will be put into one row.
(See course assignment:"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.")

In [45]:
df_toronto = df_toronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

Neighbourhoods that are not assigned, but do have a borough will be the same as the borough.
(See course assignment: "If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.")

In [46]:
df_toronto.loc[df_toronto['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_toronto['Borough']
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Check the shape of the dataframe:

In [47]:
df_toronto.shape

(103, 3)

## Final assignment part 2: adding geospatial data

The second part of the data science capstone assignment in week 3 is about adding the correct geospatial data to the Toronto neighbourhoods table. 

In [48]:
geospatial_url = requests.get('http://cocl.us/Geospatial_data').content
df_geospatial = pd.read_csv("http://cocl.us/Geospatial_data", header = 0)

df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Since the column name for the postcode in the Toronto Dataframe df_toronto is "postcode" and in the the geospatial DataFrame df_geospatial it is "Postal Code", I will rename the column accordingly to be able to merge the two dataframes.

In [49]:
df_geospatial.columns = ['Postcode', 'Latitude', 'Longitude']
df_final = pd.merge(df_toronto, df_geospatial, on='Postcode')
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
